In [ ]:
# GENERAL IMPORTS

import pandas as pd
import numpy as np
import scipy as sp

import plotly.offline as py
import plotly.graph_objs as go
import plotly
plotly.offline.init_notebook_mode(connected=True)

import ipywidgets

import pprint
import random
import datetime
import math
import bisect
import time
from os import listdir
from os.path import isfile, join

In [ ]:
class DataLibrary:
    def __init__(self, filename):
        self.filename = filename
        #self.df = pd.DataFrame()
        self.df = pd.read_csv('DataLibrary/' + self.filename, index_col = 0)
        
    def get_data(self, date, instrument, data_name):
        key = self.keygen(date, instrument)
        try:
            data = self.df[data_name].loc[key]
        except:
            self.regen(date, instrument)
            data = self.df[data_name].loc[key]
        
        if not data or np.isnan(data):
            self.regen(date, instrument)
            data = self.df[data_name].loc[key]
        
        return data
    
    def regen(self, date, instrument):
        key = self.keygen(date, instrument)
        data_names = ['Open', 'High', 'Low', 'Close', 'Volume', 'FirstMinClose']
        
        minutedatadir = r'''C:\ProgramData\Kibot Agent\Data\SP500_1'''
        minutecolnames = ['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume']
        pricedata = pd.read_csv(join(minutedatadir, instrument+'.txt'), names=minutecolnames)
        pricedata['DateTime'] = (pricedata['Date']+pricedata['Time']).map(lambda x: datetime.datetime(int(x[6:10]), int(x[0:2]), int(x[3:5]), int(x[10:12]), int(x[13:15])))
        pricedata = pricedata[[(dt >= datetime.datetime(date.year, date.month, date.day, 9, 30, 0)) and (dt <= datetime.datetime(date.year, date.month, date.day, 16, 0, 0)) for dt in pricedata['DateTime']]].copy()

        for data_name in data_names:
            try:
                temp = self.df[data_name]
            except:
                self.df[data_name] = [None] * len(self.df)

        try:
            temp = self.df.loc[key]
        except:
            new_row = {}
            for data_name in data_names:
                new_row[data_name] = None

            to_append = pd.DataFrame(new_row, index=[key])
            self.df = self.df.append(to_append)

        self.df['Open'][key] = pricedata['Open'].iloc[0]
        self.df['Close'][key] = pricedata['Close'].iloc[-1]
        self.df['High'][key] = np.max(pricedata['High'])
        self.df['Low'][key] = np.min(pricedata['Low'])
        self.df['Volume'][key] = np.sum(pricedata['Volume'])
        self.df['FirstMinClose'][key] = pricedata['Close'].iloc[0]
        
    def remove_col(self, data_name):
        self.df = self.df.drop(columns=[data_name])
        
    def keygen(self, date, instrument):
        return str(date) + str(instrument)
        
    def close(self):
        timestr = time.strftime("_%Y%m%d-%H%M%S")
        self.df.to_csv('DataLibrary/' + self.filename)
        self.df.to_csv('DataLibrary/Backups/' + self.filename[:-4] + timestr + '.txt')

In [ ]:
# dl = DataLibrary('data_library.txt')
# #dl.remove_col('Date')
# dl.get_data(datetime.date(2018,1,4), 'SHLD', 'Volume')
# dl.close()

In [ ]:
daydatadir = r'''C:\ProgramData\Kibot Agent\Data\SP500_day'''
files = [f for f in listdir(daydatadir) if isfile(join(daydatadir, f))]
instruments = [f[0:-4] for f in files]

colnames = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
daypricedata = {}
for f in instruments:
    df = pd.read_csv(join(daydatadir, f+'.txt'), names=colnames)
    df['Date'] = df['Date'].map(lambda x: datetime.date(int(x[6:10]), int(x[0:2]), int(x[3:5])))
    df['Gap'] = np.insert(np.array(df['Open'][1:])-np.array(df['Close'][:-1]),0,0)
    df = df.set_index('Date')
    daypricedata[f] = df

In [ ]:
def get_gap_instruments(param_dict):
    start_date = param_dict['Start Date']
    end_date = param_dict['End Date']
    
    num_instruments = 1
    max_gap_instruments = {}
    gaps = {}

    for ii in range((end_date - start_date).days):
        date = start_date + datetime.timedelta(days=ii)
        max_abs_gap = 0
        max_gap_instrument = None
        for instrument in instruments:
            if date in daypricedata[instrument].index:
                gap = daypricedata[instrument].loc[date]['Gap']/daypricedata[instrument].loc[date]['Open']
                abs_gap = abs(gap)
                if abs_gap > max_abs_gap:
                    max_gap = gap
                    max_abs_gap = abs_gap
                    max_gap_instrument = instrument
        if max_gap_instrument:
            max_gap_instruments[date] = max_gap_instrument
            gaps[date] = max_gap
            print(date, max_gap_instrument, max_gap)
            
    return max_gap_instruments, gaps

In [ ]:
# idx = [ii for ii in range(len(max_gap_instruments))]
# random.shuffle(idx)

# rand_dates = list(max_gap_instruments.keys())
# rand_instruments = list(max_gap_instruments.values())

# for ii in range(len(rand_dates)):
#     max_gap_instruments[rand_dates[ii]] = rand_instruments[idx[ii]]

In [ ]:
def gap_trading_strategy(portfolio, data_dict, param_dict):
    minutedatadir = r'''C:\ProgramData\Kibot Agent\Data\SP500_1'''
    minutecolnames = ['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    
    max_gap_instruments = data_dict['max_gap_instruments']
    gaps = data_dict['gaps']
    
    start_date = param_dict['Start Date']
    end_date = param_dict['End Date']
    
    for ii in range((end_date - start_date).days):
        date = start_date + datetime.timedelta(days=ii)
        
        try:
            instrument = max_gap_instruments[date]
            gap = gaps[date]
        except KeyError:
            instrument = None
            
        if instrument:
            try:
                pricedata = pd.read_csv(join(minutedatadir, instrument+'.txt'), names=minutecolnames)
                pricedata['DateTime'] = (pricedata['Date']+pricedata['Time']).map(lambda x: datetime.datetime(int(x[6:10]), int(x[0:2]), int(x[3:5]), int(x[10:12]), int(x[13:15])))
                pricedata = pricedata[[(dt >= datetime.datetime(date.year, date.month, date.day, 9, 30, 0)) and (dt <= datetime.datetime(date.year, date.month, date.day, 16, 0, 0)) for dt in pricedata['DateTime']]].copy()
                portfolio.positions[instrument] = 0
                print(date, instrument)
            except:
                continue
            
            if not pricedata.index.empty and len(pricedata) > 1:
                open_price = pricedata['Open'].iloc[0]
                for index in range(len(pricedata)):
                    row = pricedata.iloc[index]

                    #if index == len(pricedata)-1 or (np.sign(row['Close'] - open_price) != np.sign(portfolio.positions[instrument]) and index != 0):
                    if index == len(pricedata)-1:
                        if portfolio.positions[instrument] < 0:
                            portfolio.buy(instrument, -portfolio.positions[instrument], row)
                        elif portfolio.positions[instrument] > 0:
                            portfolio.sell(instrument, portfolio.positions[instrument], row)
                    elif index == 0:
                        if row['Close'] > open_price:
                        #if np.sign(gap) < 0:
                            if portfolio.positions[instrument] <= 0:
                                quantity = np.floor(10000/row['Open'])
                                portfolio.ask_price_key = 'Mean'
                                portfolio.bid_price_key = 'Mean'
                                portfolio.buy(instrument, -portfolio.positions[instrument] + quantity, row)
                                portfolio.ask_price_key = 'Close'
                                portfolio.bid_price_key = 'Close'
                        elif row['Close'] < open_price:
                        #elif np.sign(gap) > 0:
                            if portfolio.positions[instrument] >= 0:
                                quantity = np.floor(10000/row['Open'])
                                portfolio.ask_price_key = 'Mean'
                                portfolio.bid_price_key = 'Mean'
                                portfolio.sell(instrument, portfolio.positions[instrument] + quantity, row)
                                portfolio.ask_price_key = 'Close'
                                portfolio.bid_price_key = 'Close'
#                     elif row['Close'] > open_price:
#                         if portfolio.positions[instrument] <= 0:
#                             quantity = np.floor(10000/row['Close'])
#                             portfolio.buy(instrument, -portfolio.positions[instrument] + quantity, row)
#                     elif row['Close'] < open_price:
#                         if portfolio.positions[instrument] >= 0:
#                             quantity = np.floor(10000/row['Close'])
#                             portfolio.sell(instrument, portfolio.positions[instrument] + quantity, row)

                    portfolio.update(instrument, row)
    
    portfolio.portfolio_history = pd.DataFrame(portfolio.portfolio_history)
    portfolio.transaction_history = pd.DataFrame(portfolio.transaction_history)                

In [ ]:
def gap_trading_strategy2(portfolio, data_dict, param_dict):
    minutedatadir = r'''C:\ProgramData\Kibot Agent\Data\SP500_1'''
    minutecolnames = ['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    
    max_gap_instruments = data_dict['max_gap_instruments']
    gaps = data_dict['gaps']
    
    start_date = param_dict['Start Date']
    end_date = param_dict['End Date']
    dl = DataLibrary('data_library.txt')

    for ii in range((end_date - start_date).days):
        date = start_date + datetime.timedelta(days=ii)
        
        try:
            instrument = max_gap_instruments[date]
            gap = gaps[date]
        except KeyError:
            instrument = None
            
        if instrument:
#             try:
                
#                 pricedata = dl.get_data(datetime.date(2018,1,4), instrument, 'Open')
#                 print(date, instrument)
#             except:
#                 continue

            try:
                open_price = dl.get_data(date, instrument, 'Open')
                close_price = dl.get_data(date, instrument, 'Close')
                first_min_close_price = dl.get_data(date, instrument, 'FirstMinClose')
                transaction_price = dl.get_data(date, instrument, 'FirstMinClose')
                open_transaction_dt = datetime.datetime(date.year, date.month, date.day, 9, 31, 0)
                close_transaction_dt = datetime.datetime(date.year, date.month, date.day, 15, 59, 0)
            except:
                continue

            if first_min_close_price > open_price:
            #if np.sign(gap) < 0:
                quantity = np.floor(10000/transaction_price)
                portfolio.buy(instrument, quantity, transaction_price, open_transaction_dt)
                portfolio.sell(instrument, quantity, close_price, close_transaction_dt)
            elif first_min_close_price < open_price:
            #elif np.sign(gap) > 0:
                quantity = np.floor(10000/transaction_price)
                portfolio.sell(instrument, quantity, transaction_price, open_transaction_dt)
                portfolio.buy(instrument, quantity, close_price, close_transaction_dt)

            #portfolio.update(instrument, row)
    
    portfolio.portfolio_history = pd.DataFrame(portfolio.portfolio_history)
    portfolio.transaction_history = pd.DataFrame(portfolio.transaction_history)
    dl.close()

In [ ]:
class Portfolio:
    def __init__(self, param_dict):
        
        self.param_dict = param_dict
        self.starting_cash = param_dict['Starting Cash']
        self.cash = self.starting_cash
        self.positions = {}
        self.commission = param_dict['Commission']

        self.ask_price_key = 'Close'
        self.bid_price_key = 'Close'
        
        self.net_value = 0
        self.portfolio_history = []
        self.transaction_history = []
        
    def update(self, instrument, price, date_time):
        new_net_value = self.cash + self.positions[instrument] * price
        delta_net_value = round(100*(new_net_value - self.net_value))/100
        self.net_value = new_net_value
        if delta_net_value < -400:
            print("Big loss of " + str(delta_net_value) + " on " + str(date_time) + ", " + str(instrument))
        if delta_net_value > 400:
            print("Big win of " + str(delta_net_value) + " on " + str(date_time) + ", " + str(instrument))
            
        self.portfolio_history.append(
            {
                "DateTime": date_time,
                "Cash": self.cash,
                "Position": self.positions,
                "Net Value": self.net_value
            }
        )
        
    def buy(self, instrument, quantity, price, date_time):
            
        cost = quantity*price + self.commission
        self.cash -= cost

        if not instrument in self.positions.keys():
            self.positions[instrument] = 0
        self.positions[instrument] += quantity

        self.cash = round(self.cash*100)/100

        self.transaction_history.append(
            {
                "DateTime": date_time,
                "Instrument" : instrument,
                "Price": price,
                "Quantity": quantity
            }
        )
        self.update(instrument, price, date_time)
        
    def sell(self, instrument, quantity, price, date_time):
        
        self.cash += quantity*price - self.commission

        if not instrument in self.positions.keys():
            self.positions[instrument] = 0
        self.positions[instrument] -= quantity

        self.cash = round(self.cash*100)/100

        self.transaction_history.append(
            {
                "DateTime": date_time,
                "Instrument" : instrument,
                "Price": price,
                "Quantity": -quantity
            }
        )
        self.update(instrument, price, date_time)
        
    def logging_dict(self):
        max_net_value = self.portfolio_history["Net Value"][0]
        max_drawdown = 0
        max_drawdown_percentage = 0
        for ii in range(len(self.portfolio_history)):
            max_net_value = max(max_net_value, self.portfolio_history["Net Value"][ii])
            max_drawdown = max(max_drawdown, max_net_value - self.portfolio_history["Net Value"][ii])
            max_drawdown_percentage = max(max_drawdown_percentage, 1 - self.portfolio_history["Net Value"][ii] / max_net_value)
            
        d = {
            'param_dict' : self.param_dict,
            'Starting Cash' : self.starting_cash,
            'Commission' : self.commission,
            'Profit' : self.net_value-self.starting_cash,
            'Annualized Return (Compounding)' : (self.net_value / self.starting_cash)**(365 / (param_dict['End Date']-param_dict['Start Date']).days) - 1,
            'Annualized Profit (Linear)' : (self.net_value - self.starting_cash)*(365 / (param_dict['End Date']-param_dict['Start Date']).days),
            'Number of Trades' : len(self.transaction_history),
            'Max Drawdown' : max_drawdown,
            'Max Drawdown %' : max_drawdown / max_net_value * 100
        }
        
        return d

In [ ]:
def log(log_file_path, param_dict, portfolio, notes):
    log_file = open(log_file_path, 'w')
    pp = pprint.PrettyPrinter(indent=4, stream=log_file)
    pp.pprint(portfolio.logging_dict())
    pp.pprint({'Notes' : notes})
    log_file.flush()
    log_file.close()

In [ ]:
timestr = time.strftime("%Y%m%d-%H%M%S")
log_path = 'logs/AlgoTrading1/'
log_file_name = timestr + ".txt"
log_file_path = log_path + log_file_name
notes = input("Enter notes for this run: ")

param_dict = {
    'Starting Cash' : 50000,
    'Commission' : 5,
    'Start Date': datetime.date(2018,1,1),
    'End Date': datetime.date(2019,1,1)
    
}
max_gap_instruments, gaps = get_gap_instruments(param_dict)
data_dict = {
    'max_gap_instruments': max_gap_instruments,
    'gaps': gaps
}

portfolio = Portfolio(param_dict)
#gap_trading_strategy(portfolio, data_dict, param_dict)
gap_trading_strategy2(portfolio, data_dict, param_dict)

log(log_file_path, param_dict, portfolio, notes)

In [ ]:
class ProfitCharts():
    def __init__(self, portfolio):
        self.portfolio = portfolio
        cash = portfolio.portfolio_history['Cash']
        net_value = portfolio.portfolio_history['Net Value']
        
        x_vals = [dt.strftime("%y/%m/%d, %H:%M:%S") for dt in portfolio.portfolio_history['DateTime']]
        tick_vals = [ii for ii in range(len(x_vals)) if 
                                 portfolio.portfolio_history['DateTime'][ii].time() == datetime.time(9, 30)]
        xaxisdatetimes = [portfolio.portfolio_history['DateTime'][ii] for ii in tick_vals]
        tick_text = [dt.strftime(' %y/%m/%d ') if dt.time() == datetime.time(9, 30)
                                 else dt.strftime(' %H:%M ') for dt in xaxisdatetimes]

        data1 = [ dict(
                type = 'scatter',
                x = x_vals,
                y = cash,
                name = 'Cash'),

                dict(
                type = 'scatter',
                x = x_vals,
                y = net_value - cash,
                name = 'Position'),

                dict(
                type = 'scatter',
                x = x_vals,
                y = net_value,
                name = 'Net Value')
               ]
        
        layout1 = dict(
                title = 'Profit Chart',
                xaxis = dict(
                    title = dict(
                        text = 'DateTime'
                    ),
                    type = 'category', 
                    categoryorder = 'category ascending',
                    tickvals = tick_vals,
                    ticktext = tick_text
                ),
                yaxis = dict(
                    title = dict(
                        text = 'Value'
                    )
                ))
        
        self.chart1 = go.FigureWidget( data=data1, layout=layout1)
        
        data2 = [
                dict(
                type = 'scatter',
                x = x_vals,
                y = [nv - net_value[0] for nv in net_value],
                name = 'Profit')
               ]
        
        layout2 = dict(
                title = 'Profit Chart',
                xaxis = dict(
                    title = dict(
                        text = 'DateTime'
                    ),
                    type = 'category', 
                    categoryorder = 'category ascending',
                    tickvals = tick_vals,
                    ticktext = tick_text
                ),
                yaxis = dict(
                    title = dict(
                        text = 'Profit'
                    )
                ))
        
        self.chart2 = go.FigureWidget( data=data2, layout=layout2)        
        # DEFINE DISPLAY CONFIGURATION
        self.display = ipywidgets.VBox([self.chart1, self.chart2])

In [ ]:
portfolio.transaction_history

In [ ]:
profit_charts = ProfitCharts(portfolio)
profit_charts.display